# Projeto 3 - Google Play Store Apps

Ciências dos Dados

**1.Arthur Ventrilho**

**2.Gabriel Zanetti**

**3.Roger Pina**

### Etapa 1: Importando e colocando as funções que serão utilizadas

In [5]:
%matplotlib inline
import pandas as pd
import numpy as np
import math
import mpmath
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import norm, probplot, t

In [6]:
# Função utilitária para fazer a regressão com constante adicionada
def regress_cp(X,Y):
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp)
    results = model.fit()
    return results

def regress(X,Y):
    model = sm.OLS(Y,X)
    results = model.fit()
    return results

In [7]:
#Função que transforma colunas categóricas em colunas quantitativas
def dummify(data, column_name):
    """
        Converte a coluna column_name em dummies / one-hot e as adiciona ao dataframe
        retorna uma copia do  df original *sem* a coluna que foi dummified
    """
    df = data.copy()
    # Nota: se você for um econometrista, pode se interessar por usar a opção dropFirst = True na get_dummies
    df2 = pd.concat([df.drop(column_name, axis=1), pd.get_dummies(data[column_name], prefix=column_name + "_")], axis=1)
    return df2

In [12]:
#Rating como saída
data = pd.read_csv('googleplaystore.csv')
data = data.dropna(axis=0)

#Install como saída
data_ = pd.read_csv('googleplaystore.csv')
data_ = data_.dropna(axis=0)

FileNotFoundError: File b'googleplaystore.csv' does not exist

### Etapa 2: Arrumando os dados

In [9]:
del data['Genres']
del data['Last Updated']
del data['Current Ver']
del data['Android Ver']

del data_['Genres']
del data_['Last Updated']
del data_['Current Ver']
del data_['Android Ver']

NameError: name 'data' is not defined

In [10]:
#Tranformando Installs em quant
deletar = ['+', ',']
Installs_novo = []
Installs_novo_ = []

for frase in data['Installs']:
    for c in deletar:
        frase = frase.replace(c , "")
    Installs_novo.append(frase)
Installs_novo
data['Installs'] = pd.Series((Installs_novo), index = data.index)

for frase in data_['Installs']:
    for c in deletar:
        frase = frase.replace(c , "")
    Installs_novo_.append(frase)
Installs_novo_
data_['Installs'] = pd.Series((Installs_novo_), index = data_.index)

NameError: name 'data' is not defined

In [11]:
#Deixando Price em quant
deletar = ['$']
Price_novo = []
Price_novo_ = []

for frase in data['Price']:
    for c in deletar:
        frase = frase.replace(c , "")
    Price_novo.append(frase)
data['Price'] = pd.Series((Price_novo), index = data.index)

for frase in data_['Price']:
    for c in deletar:
        frase = frase.replace(c , "")
    Price_novo_.append(frase)
data_['Price'] = pd.Series((Price_novo_), index = data_.index)

NameError: name 'data' is not defined

In [ ]:
data = data.drop(data[data.Size == "Varies with device"].index)
data_ = data_.drop(data_[data_.Size == "Varies with device"].index)

def retira_letras(DF, coluna):
    listaA = []
        
    if coluna == 'Size':
        for linha in DF[coluna]:
            if linha[-1] == 'k':
                listaA.append(int(float(linha[:-1]) * 1000))
            elif linha[-1] == 'M':
                listaA.append(int(float(linha[:-1]) * 1000000))
    DF[coluna] = pd.Series((listaA), index = DF.index)


retira_letras(data, 'Size')
retira_letras(data_, 'Size')

In [ ]:
data.head(5)

In [ ]:
def transforma_em_numero(DF):
    colunas_int = ['Reviews', 'Size', 'Installs']
    colunas_ft = ['Rating', 'Price']
    
    for coluna in colunas_ft:
        listaft = []
        for linha in DF[coluna]:
            listaft.append(float(linha))
        DF[coluna] = pd.Series((listaft), index = DF.index)
    
    for coluna in colunas_int:
        lista = []
        for linha in DF[coluna]:
            lista.append(int(linha))
        DF[coluna] = pd.Series((lista), index = DF.index)
        
transforma_em_numero(data)
transforma_em_numero(data_)

In [ ]:
#Declarando Categórico para a saída Rating
data.Category = data.Category.astype('category')
data.Type = data.Type.astype('category')
data['Content Rating'] = data['Content Rating'].astype('category')

In [ ]:
#Declarando Categórico para a saída Install
data_.Category = data_.Category.astype('category')
data_.Type = data_.Type.astype('category')
data_['Content Rating'] = data_['Content Rating'].astype('category')

In [ ]:
#Declarando Quantitativas
quantitativas = ['Rating', 'Reviews', 'Size', 'Installs', 'Price']

### Etapa 3: Rating como saída

#### 3.1: Regressão Linear somente com as colunas quantitativas 

In [ ]:
#Separando a coluna que será a saída
quant = quantitativas.copy()
quant.remove('Rating')

In [ ]:
#Fazendo a regressão
results = regress_cp(data.loc[:, quant], data.Rating)
results.summary()

Analisando os dados podemos concluir que os resíduos da nossa regressão linear não seguem uma distribuição normal, isso foi visto através da Prob(Omnibus) e Prob(JB) que foram altas, com isso não podemos confiar no p-value. 

In [ ]:
#Justificativa do que foi afirmado acima
probplot(results.resid, dist="norm", plot=plt);

#### 3.2: Tirando a constante da regressão linear

In [ ]:
#Fazendo a regressão
results = regress(data.loc[:, quant], data.Rating)
results.summary()

#### 3.3: Transformando as colunas categóricas em quantitativas

In [ ]:
#Covertendo Category
cat = dummify(data, "Category")

In [ ]:
quant.extend(['Category__ART_AND_DESIGN', 'Category__AUTO_AND_VEHICLES','Category__BEAUTY', 'Category__BOOKS_AND_REFERENCE',
       'Category__BUSINESS', 'Category__COMICS', 'Category__COMMUNICATION','Category__DATING', 'Category__EDUCATION', 'Category__ENTERTAINMENT',
       'Category__EVENTS', 'Category__FAMILY', 'Category__FINANCE','Category__FOOD_AND_DRINK', 'Category__GAME',
       'Category__HEALTH_AND_FITNESS', 'Category__HOUSE_AND_HOME','Category__LIBRARIES_AND_DEMO', 'Category__LIFESTYLE',
       'Category__MAPS_AND_NAVIGATION', 'Category__MEDICAL','Category__NEWS_AND_MAGAZINES', 'Category__PARENTING',
       'Category__PERSONALIZATION', 'Category__PHOTOGRAPHY','Category__PRODUCTIVITY', 'Category__SHOPPING', 'Category__SOCIAL',
       'Category__SPORTS', 'Category__TOOLS', 'Category__TRAVEL_AND_LOCAL','Category__VIDEO_PLAYERS', 'Category__WEATHER'])

In [ ]:
#Fazendo a regressão somente com as categóricas da coluna "Category"
results1 = regress(cat.loc[:, quant], cat.Rating)
results1.summary()

In [ ]:
#Covertendo Type
tp = dummify(cat, "Type")
tp_ = dummify(data, "Type")

In [ ]:
#Separando a coluna que será a saída
quants = quantitativas.copy()
quants.remove('Rating')

In [ ]:
quant.extend(['Type__Free','Type__Paid'])
quants.extend(['Type__Free','Type__Paid'])

In [ ]:
#Adicionando as colunas categóricas da coluna "Type" na regressão
results2 = regress(tp.loc[:, quant], tp.Rating)
results2.summary()

In [ ]:
#Fazendo a regressão somente com as categóricas da coluna "Type"
results4 = regress(tp_.loc[:, quants], tp_.Rating)
results4.summary()

In [ ]:
#Covertendo Content Rating
df = dummify(tp, "Content Rating")

In [ ]:
quant.extend(['Content Rating__Adults only 18+', 'Content Rating__Everyone','Content Rating__Everyone 10+', 'Content Rating__Mature 17+',
       'Content Rating__Teen', 'Content Rating__Unrated'])

In [ ]:
#Refazendo a regressao após adicionar todas as categóricas que foram transformadas em quant
results3 = regress(df.loc[:, quant], data.Rating)
results3.summary()

In [ ]:
quant.remove('Installs')
quant.remove('Category__DATING')
quant.remove('Category__MAPS_AND_NAVIGATION')
quant.remove('Category__TOOLS')

results = regress(df.loc[:, quant], data.Rating)
results.summary()

### Etapa 4: Install como saída


#### 4.1: Regressão Linear somente com as colunas quantitativas 

In [ ]:
#Declarando Quantitativas
quantitativas = ['Rating', 'Reviews', 'Size', 'Installs', 'Price']

In [ ]:
#Separando a coluna que será a saída
quant_ = quantitativas.copy()
quant_.remove('Installs')

In [ ]:
#Fazendo a regressão
resultado = regress_cp(data_.loc[:, quant_], data_.Installs)
resultado.summary()

In [ ]:
#Analisando graficamente
probplot(resultado.resid, dist="norm", plot=plt);

O gráfico acima nos permite afirmar que os resíduos dessa regressão linear seguem uma distribuição normal

#### 4.2: Tirando a constante da regressão linear

In [ ]:
#Fazendo a regressão
resultado1 = regress(data_.loc[:, quant_], data_.Installs)
resultado1.summary()

OBS: Observamos que essa regressão é impossível de ser feita dado que o Re2 deu um número negativo

In [ ]:
data_["logInstalls"] = np.log(data_.logInstalls)

In [ ]:
#Fazendo a regressão
resultado_log = regress(data_.loc[:, quant_], data_.logInstalls)
resultado_log.summary()